**Nom Etudiant :**  ADAMA

**Prenom Etudiant:**  BOCOUM

**Classe :** ING3 INFO IPLSL

**Date limite de dépot : 31/08/2022**


# Travail à Faire:
Télécharger le Datasets sur le lien Drive :  https://drive.google.com/file/d/1tuV0lyb2uWvOInodVTohpdCiAV5FrBX-/view?usp=sharing 

Repondre les questions ci-dessous avec le maximum de precisions et de détails.   
Remplir `FILL_IN` avec les methodes qui correspondent à la réponse adéquate


### Revenus des achats
1. Extraire les revenus d'achat pour chaque événement
2. Filtrer les événements dont le revenu n'est pas nul
3. Vérifiez quels sont les types d'événements qui génèrent des revenus
4. Supprimez la colonne inutile

### Revenus par Traffic
Obtenir les 3 sources de trafic générant le revenu total le plus élevé.
5. Revenus cumulés par source de trafic
7. Obtenir les 3 principales sources de trafic par revenu total
6. Nettoyer les colonnes de revenus pour avoir deux décimales


##### Methods
- DataFrame: `select`, `drop`, `withColumn`, `filter`, `dropDuplicates`,  `groupBy`, `sort`, `limit`
- Column: `isNotNull`, `alias`, `desc`, `cast`, `operators`

In [27]:
%scala
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder.
                        appName("Projet Adama Bocoum").
                        config("spark.ui.port", "0").
                        master("yarn").
                        getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2d2c498a


In [28]:
%scala
val events = spark.read.json("datasets/events/events.json")

events: org.apache.spark.sql.DataFrame = [device: string, ecommerce: struct<purchase_revenue_in_usd: double, total_item_quantity: bigint ... 1 more field> ... 8 more fields]


In [29]:
val eventsPath = "mnt/training/ecommerce/events/events.parquet"

eventsPath: String = mnt/training/ecommerce/events/events.parquet


In [30]:
val eventsDF=events.
    write.
    mode("overwrite").
    option("compression", "none").
    format("parquet").
    save(eventsPath)

eventsDF: Unit = ()


In [32]:
%scala
// Read in the parquet file created above
// Parquet files are self-describing so the schema is preserved
// The result of loading a Parquet file is also a DataFrame
val eventsDF = spark.read.parquet(eventsPath)
eventsDF.show()

+-------+---------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+
| device|ecommerce|event_name|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|
+-------+---------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+
|  macOS|     [,,]|  warranty|        1593878899217692|1593878946592107|      [Montrose, MI]|                  []|        google|          1593878899217692|UA000000107379500|
|Windows|     [,,]|     press|        1593876662175340|1593877011756535|   [Northampton, MA]|                  []|        google|          1593876662175340|UA000000107359357|
|  macOS|     [,,]|  add_item|        1593878792892652|1593878815459100|       [Salinas, CA]|[[, M_STAN_T, Sta...|       yout

eventsDF: org.apache.spark.sql.DataFrame = [device: string, ecommerce: struct<purchase_revenue_in_usd: double, total_item_quantity: bigint ... 1 more field> ... 8 more fields]


### 1. Extraire les revenus d'achat pour chaque événement
Ajouter une nouvelle colonne **`revenue`** en faisant l'extration de **`ecommerce.purchase_revenue_in_usd`**

In [34]:
%scala
import org.apache.spark.sql.functions.{lit, lpad,col}

val revenueDF= eventsDF.withColumn("revenue",col("ecommerce.purchase_revenue_in_usd"))
revenueDF.show()

+-------+---------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
| device|ecommerce|event_name|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|revenue|
+-------+---------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|  macOS|     [,,]|  warranty|        1593878899217692|1593878946592107|      [Montrose, MI]|                  []|        google|          1593878899217692|UA000000107379500|   null|
|Windows|     [,,]|     press|        1593876662175340|1593877011756535|   [Northampton, MA]|                  []|        google|          1593876662175340|UA000000107359357|   null|
|  macOS|     [,,]|  add_item|        1593878792892652|1593878815459100|       [Salin

import org.apache.spark.sql.functions.{lit, lpad, col}
revenueDF: org.apache.spark.sql.DataFrame = [device: string, ecommerce: struct<purchase_revenue_in_usd: double, total_item_quantity: bigint ... 1 more field> ... 9 more fields]


In [35]:
//Colonne Revenue Creee ci-dessous derniere Ligne
 revenueDF.printSchema()

root
 |-- device: string (nullable = true)
 |-- ecommerce: struct (nullable = true)
 |    |-- purchase_revenue_in_usd: double (nullable = true)
 |    |-- total_item_quantity: long (nullable = true)
 |    |-- unique_items: long (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_previous_timestamp: long (nullable = true)
 |-- event_timestamp: long (nullable = true)
 |-- geo: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- coupon: string (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- item_name: string (nullable = true)
 |    |    |-- item_revenue_in_usd: double (nullable = true)
 |    |    |-- price_in_usd: double (nullable = true)
 |    |    |-- quantity: long (nullable = true)
 |-- traffic_source: string (nullable = true)
 |-- user_first_touch_timestamp: long (nullable = true)

### 2. Filtrer les événements dont le revenu n'est pas null

In [36]:
%scala
// TODO
val purchasesDF = revenueDF.select("event_name","revenue","traffic_source").filter(revenueDF("revenue").isNotNull)
purchasesDF.show()

+----------+-------+--------------+
|event_name|revenue|traffic_source|
+----------+-------+--------------+
|  finalize| 1195.0|        google|
|  finalize| 1045.0|      facebook|
|  finalize|  595.0|        google|
|  finalize| 2290.0|        google|
|  finalize|  945.0|      facebook|
|  finalize|  595.0|        google|
|  finalize|  945.0|        direct|
|  finalize| 1095.0|     instagram|
|  finalize| 1045.0|        direct|
|  finalize| 1045.0|     instagram|
|  finalize| 1045.0|     instagram|
|  finalize| 1640.0|      facebook|
|  finalize|  654.0|        google|
|  finalize| 1045.0|        direct|
|  finalize| 1795.0|      facebook|
|  finalize| 1195.0|        google|
|  finalize| 1195.0|        google|
|  finalize| 1128.6|         email|
|  finalize| 1075.5|         email|
|  finalize|  595.0|      facebook|
+----------+-------+--------------+
only showing top 20 rows



purchasesDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [event_name: string, revenue: double ... 1 more field]


### 3. Vérifiez quels sont les types d'événements qui génèrent des revenus
Trouvez des valeurs **`event_name`** uniques dans **`purchasesDF`**. Il y a deux façons de faire :
- Sélectionnez "event_name" et recupérer les enregistrements distincts
- Supprimez les enregistrements en double en fonction de "event_name" uniquement

In [37]:
%scala
// TODO
val distinctDF = purchasesDF.distinct()
distinctDF.show

+----------+-------+--------------+
|event_name|revenue|traffic_source|
+----------+-------+--------------+
|  finalize|  940.5|         email|
|  finalize| 1195.0|     instagram|
|  finalize|  535.5|         email|
|  finalize| 1195.0|         email|
|  finalize| 1045.0|        direct|
|  finalize| 1274.0|      facebook|
|  finalize|  595.0|        google|
|  finalize| 2290.0|        google|
|  finalize|   79.0|      facebook|
|  finalize| 1045.0|         email|
|  finalize| 1045.0|        google|
|  finalize| 1204.0|      facebook|
|  finalize|   59.0|        google|
|  finalize| 1195.0|        google|
|  finalize|  945.0|        google|
|  finalize|  985.5|         email|
|  finalize| 1814.0|        direct|
|  finalize| 1995.0|       youtube|
|  finalize|  921.6|         email|
|  finalize| 1075.5|         email|
+----------+-------+--------------+
only showing top 20 rows



distinctDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [event_name: string, revenue: double ... 1 more field]


### 4. Supprimez la colonne inutile
Puisqu'il n'y a qu'un seul type d'événement, supprimez **`event_name`** de **`purchasesDF`**.

In [38]:
%scala
// TODO

val cleanDF = purchasesDF.drop("event_name")
cleanDF.show()

+-------+--------------+
|revenue|traffic_source|
+-------+--------------+
| 1195.0|        google|
| 1045.0|      facebook|
|  595.0|        google|
| 2290.0|        google|
|  945.0|      facebook|
|  595.0|        google|
|  945.0|        direct|
| 1095.0|     instagram|
| 1045.0|        direct|
| 1045.0|     instagram|
| 1045.0|     instagram|
| 1640.0|      facebook|
|  654.0|        google|
| 1045.0|        direct|
| 1795.0|      facebook|
| 1195.0|        google|
| 1195.0|        google|
| 1128.6|         email|
| 1075.5|         email|
|  595.0|      facebook|
+-------+--------------+
only showing top 20 rows



cleanDF: org.apache.spark.sql.DataFrame = [revenue: double, traffic_source: string]


### 5. Revenus cumulés par source de trafic
- Grouper par **`traffic_source`**
- Obtenir la somme de **`revenue`** comme **`total_rev`**
- Obtenir la moyenne de **`revenue`** comme **`avg_rev`**

N'oubliez pas d'importer toutes les fonctions intégrées nécessaires.

In [39]:
%scala
import org.apache.spark.sql.functions.{avg,sum}

val trafficDF = cleanDF.groupBy("traffic_source").agg(sum("revenue").as("total_rev"),avg("revenue").as("avg_rev"))
trafficDF.show()

+--------------+---------+-----------------+
|traffic_source|total_rev|          avg_rev|
+--------------+---------+-----------------+
|     instagram|   8160.0|           1020.0|
|        direct|   9129.0|         1141.125|
|       youtube|   3040.0|           1520.0|
|         email|  36935.4|998.2540540540541|
|      facebook|  12952.0|996.3076923076923|
|        google|  28936.0|964.5333333333333|
+--------------+---------+-----------------+



import org.apache.spark.sql.functions.{avg, sum}
trafficDF: org.apache.spark.sql.DataFrame = [traffic_source: string, total_rev: double ... 1 more field]


### 2. Obtenir les trois principales sources de trafic par revenu total
- Trier par **`total_rev`** dans l'ordre décroissant
- Se limiter aux trois premières lignes

In [40]:
%scala
//TODO
val topTrafficDF = trafficDF.orderBy(desc("total_rev")).limit(3)
topTrafficDF.show()

+--------------+---------+-----------------+
|traffic_source|total_rev|          avg_rev|
+--------------+---------+-----------------+
|         email|  36935.4|998.2540540540541|
|        google|  28936.0|964.5333333333333|
|      facebook|  12952.0|996.3076923076923|
+--------------+---------+-----------------+



topTrafficDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [traffic_source: string, total_rev: double ... 1 more field]


### 3. Limitez les colonnes de revenus à deux décimales pointés
- Modifier les colonnes **`avg_rev`** et **`total_rev`** pour les convertir en des nombres avec deux décimales pointés

In [41]:
%scala
import org.apache.spark.sql.functions.{bround}
// TODO
val finalDF = topTrafficDF.select(col("traffic_source"),bround(col("avg_rev"),2),bround(col("total_rev"),2))
finalDF.show()

+--------------+------------------+--------------------+
|traffic_source|bround(avg_rev, 2)|bround(total_rev, 2)|
+--------------+------------------+--------------------+
|         email|            998.25|             36935.4|
|        google|            964.53|             28936.0|
|      facebook|            996.31|             12952.0|
+--------------+------------------+--------------------+



import org.apache.spark.sql.functions.bround
finalDF: org.apache.spark.sql.DataFrame = [traffic_source: string, bround(avg_rev, 2): double ... 1 more field]
